# Translate CrowdTangle Hindi columns to English

## Installs and imports

In [ ]:
!pip install --upgrade pip
!pip install sentencepiece
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 24.8 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 18.3 MB/s eta 0:00:00
  Attempting unins

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import MarianTokenizer, MarianMTModel
from sklearn.metrics import classification_report
from collections import defaultdict
from matplotlib import pyplot as plt
from datasets import load_dataset
import torch
import numpy as np
import urllib
import pandas as pd

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
max_length = 512

## Load the `CrowdTangle` dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv (
    '/content/drive/My Drive/The-London-Story/CrowdTangle-TLS-workspace/2022-07-11-16-34-58-CEST-Historical-Report-2016-12-31--2022-07-11.csv',
    low_memory=False,  
    lineterminator='\n', 
    sep=';', 
    error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.shape

(604703, 40)

Because the size of the dataset is too large, let's choose first N values from the dataframe

In [ ]:
df = df.head(500)

In [ ]:
df.columns

Index(['account.name', 'account.handle', 'platformId', 'Page Category',
       'Page Admin Top Country', 'Page Description', 'Page Created',
       'subscriberCount', 'Followers at Posting', 'date', 'Post Created Date',
       'Post Created Time', 'type', 'totalInteraction',
       'statistics.actual.likeCount', 'statistics.actual.commentCount',
       'statistics.actual.shareCount', 'statistics.actual.loveCount',
       'statistics.actual.wowCount', 'statistics.actual.hahaCount',
       'statistics.actual.sadCount', 'statistics.actual.angryCount',
       'statistics.actual.careCount', 'Video Share Status', 'Is Video Owner?',
       'statistics.actual.videoPostViewCount',
       'statistics.actual.videoTotalViewCount',
       'statistics.actual.videoAllCrosspostsViewCount', 'Video Length',
       'postUrl', 'message', 'expandedLinks.original',
       'expandedLinks.expanded', 'imageText', 'title', 'description',
       'brandedContentSponsor.platformId', 'brandedContentSponsor.name',
 

## Data preprocessing

Replace NaN values in particular columns with empty string

In [ ]:
df.message = df.message.fillna('')
df.description = df.description.fillna('')
df.imageText = df.imageText.fillna('')

Extract particular columns and convert into lists

In [ ]:
accountname_list =  df ['account.name'].tolist()
message_list = df ['message'].tolist()
description_list = df ['description'].tolist()
imageText_list = df ['imageText'].tolist()
expandedLinks_list = df['expandedLinks.original'].tolist()

print ("Total number of entries in the dataset:", len(message_list))

Total number of entries in the dataset: 500


## Load the translation model and the tokenizer


Load the translation model

In [ ]:
src = "hi"  # source language
trg = "en"  # target language

# loading the translation model
tr_model_name = f"Helsinki-NLP/opus-mt-{src}-{trg}"
tr_model = MarianMTModel.from_pretrained(tr_model_name)
tr_model= tr_model.to(device)

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/304M [00:00<?, ?B/s]

Load the tokenizer

In [ ]:
# loading the tokenizer of the translation model
tr_tokenizer = MarianTokenizer.from_pretrained(tr_model_name)

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/813k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Define function to translate the text

In [ ]:
def translate_hi_to_en (TEXT):
  return tr_tokenizer.batch_decode (tr_model.generate(**tr_tokenizer(TEXT, truncation=True, padding=True, max_length=max_length, return_tensors="pt").to(device)),skip_special_tokens=True)[0]

## Translate the dataset

In [ ]:
# declare an empty dataframe to populate with the translations
output_df = pd.DataFrame (columns=
                         ['accountname',
                          'message', 
                          'description', 
                          'imageText', 
                          'message_translation',
                          'description_translation',
                          'imageText_translation',
                          'link'
                          ], 
                          index=range(len(accountname_list)))

 **Translate the dataset**

In [ ]:
N = len(message_list)


for i in range (N):
  
  if i % 100 == 0:
    print('Example', i+1,' of ', N)

  output_df.loc[i].accountname = accountname_list [i]
  output_df.loc[i].message = message_list [i]
  output_df.loc[i].description = description_list [i]
  output_df.loc[i].imageText = imageText_list [i]
  output_df.loc[i].message_translation = translate_hi_to_en (message_list [i])
  output_df.loc[i].description_translation = translate_hi_to_en (description_list [i])
  output_df.loc[i].imageText_translation = translate_hi_to_en (imageText_list [i])
  output_df.loc[i].link = expandedLinks_list [i]

Example 1  of  500
Example 101  of  500
Example 201  of  500
Example 301  of  500
Example 401  of  500


In [ ]:
output_df.tail()

,accountname,message,description,imageText,message_translation,description_translation,imageText_translation,link
495,Sudarshan News,मुंबई: आरे मेट्रो कार शेड के खिलाफ प्रदर्शन मे...,शिवसेना नेता आदित्य ठाकरे ने कहा कि आरे की लड़...,,Mumbai: Big thing was #Shinen dut #Auxane lead...,Shivina leader Mutharre said that Arne's war i...,But what about you?,https://www.sudarshannews.in/shivsena-leader-a...
496,Jalamsingh surana,,,जैसलमेर जैसलमेर-पोकरण भास्कर 07-07-2022 जैसलमे...,But what about you?,But what about you?,"Muller Tsermer-Permor, a 07-202 Joller-Permmer...",https://www.facebook.com/photo.php?fbid=403786...
497,Love jihad लव जिहाद,,,"जमाना बदल गया है यारों, अब ग्राहक भगवान नहीं, ...",But what about you?,But what about you?,"Guys, men have changed, no longer customer god...",https://www.facebook.com/photo.php?fbid=432360...
498,Updesh rana youth brigade,,मैं शंभू नाथ रेगर के कार्य से प्रसन्न हूं मेरा...,,But what about you?,"I'm pleased with the work of the Nöhrer, my ro...",But what about you?,https://www.facebook.com/100005952257352/posts...
499,Kapil Mishra Fans,सच्चाई कभी नही छुप सकती 👇👇👇,,Tweet Panchjanya पाज्वजन्य @epanchjanya बाटला ...,The Truth Can Never Hide,But what about you?,Tweet Panchjanya circum3a @ecudavany Povlller ...,https://www.facebook.com/photo.php?fbid=354206...


Save the translated dataset

In [ ]:
output_df.to_csv('/content/drive/MyDrive/translated_CT.csv', index=False)